# Polymarket Data Schema Mapping For Database System

### The goal of this notebook is to figure out how one can grab all data from our polymarket source and store it in a database

### Data Objects in our database

#### Polymarket Data: these come straight from polymarket, and we will add them with little additional meddling
- Event
- Market
- Series
- Tag
- Comment -> comment on a market made by a user

##### Other data we can get but I haven't decided what to do with them yet
- Position -> a position held by a user
- User -> username, user proxy wallet, 
- Trade
- Activity (trade is a subset of activity)
- Top Holders?
- Closed Positions?
- total markets a user has traded (count)
- open interest for a market
- volume for a market
- 

#### CLOB
- can get prices, histories, order books, etc
- not super useful, as the web sockets cover most of it

#### Websocket
- Market channel
    - Price change
    - tick_size_change
    - last_trade_price
    - book (many price changes over and over again)
- User Channel

#### RTDS
- Crypto prices
- comments on events


In [11]:
# GETTING POSITIONS

# url = "https://data-api.polymarket.com/positions"

# querystring = {"sizeThreshold":"1","limit":"100","sortBy":"CURRENT","sortDirection":"DESC", "user":"0xca85f4b9e472b542e1df039594eeaebb6d466bf2"}

# response = requests.get(url, params=querystring)

# print(response.json())

# GETTING TRADES BY A USER
# import requests

# url = "https://data-api.polymarket.com/trades"

# querystring = {"limit":"100","takerOnly":"true","user":"0xca85f4b9e472b542e1df039594eeaebb6d466bf2"}

# response = requests.get(url, params=querystring)

# print(response.json())

### Getting Events

In [35]:
from useful_functions import get_all_events
import json

In [36]:
events = get_all_events(True)

id
string
​
ticker
string | null
​
slug
string | null
​
title
string | null
​
subtitle
string | null
​
description
string | null
​
resolutionSource
string | null
​
startDate
string<date-time> | null
​
creationDate
string<date-time> | null
​
endDate
string<date-time> | null
​
image
string | null
​
icon
string | null
​
active
boolean | null
​
closed
boolean | null
​
archived
boolean | null
​
new
boolean | null
​
featured
boolean | null
​
restricted
boolean | null
​
liquidity
number | null
​
volume
number | null
​
openInterest
number | null
​
sortBy
string | null
​
category
string | null
​
subcategory
string | null
​
isTemplate
boolean | null
​
templateVariables
string | null
​
published_at
string | null
​
createdBy
string | null
​
updatedBy
string | null
​
createdAt
string<date-time> | null
​
updatedAt
string<date-time> | null
​
commentsEnabled
boolean | null
​
competitive
number | null
​
volume24hr
number | null
​
volume1wk
number | null
​
volume1mo
number | null
​
volume1yr
number | null
​
featuredImage
string | null
​
disqusThread
string | null
​
parentEvent
string | null
​
enableOrderBook
boolean | null
​
liquidityAmm
number | null
​
liquidityClob
number | null
​
negRisk
boolean | null
​
negRiskMarketID
string | null
​
negRiskFeeBips
integer | null
​
commentCount
integer | null
​
imageOptimized
object
Show child attributes
​
iconOptimized
object
Show child attributes
​
featuredImageOptimized
object
Show child attributes
​
subEvents
string[] | null
​
markets
object[]
Show child attributes
​
series
object[]
Show child attributes
​
categories
object[]
Show child attributes
​
collections
object[]
Show child attributes
​
tags
object[]
Show child attributes
​
cyom
boolean | null
​
closedTime
string<date-time> | null
​
showAllOutcomes
boolean | null
​
showMarketImages
boolean | null
​
automaticallyResolved
boolean | null
​
enableNegRisk
boolean | null
​
automaticallyActive
boolean | null
​
eventDate
string | null
​
startTime
string<date-time> | null
​
eventWeek
integer | null
​
seriesSlug
string | null
​
score
string | null
​
elapsed
string | null
​
period
string | null
​
live
boolean | null
​
ended
boolean | null
​
finishedTimestamp
string<date-time> | null
​
gmpChartMode
string | null
​
eventCreators
object[]
Show child attributes
​
tweetCount
integer | null
​
chats
object[]
Show child attributes
​
featuredOrder
integer | null
​
estimateValue
boolean | null
​
cantEstimate
boolean | null
​
estimatedValue
string | null
​
templates
object[]
Show child attributes
​
spreadsMainLine
number | null
​
totalsMainLine
number | null
​
carouselMap
string | null
​
pendingDeployment
boolean | null
​
deploying
boolean | null
​
deployingTimestamp
string<date-time> | null
​
scheduledDeploymentTimestamp
string<date-time> | null
​
gameStatus
string | null

#### Data to glean from events

- event_id: int = event['id'] # -> the id of the event

EVERYTHING IS NULLABLE EXCEPT FOR ID

- event_ticker: str = event['ticker'] # ticker for the event
- event_slug: str = event['slug'] # unique slug for the event
- event_title: str = event['title'] # title
- event_description: str = event['description']
- event_resolution_source: str = event['resolutionSource']
- event_creation_date: datetime = event['creationDate'] # when the event was created? Z datetime
- event_start_date: datetime = event['startDate'] # when the market starts trading? Z datetime 
- event_end_date: datetime = event['endDate'] # when the market resolves
- active: bool = event['active']
- closed: bool = event['closed']
- archived: bool = event['archived']
- restricted: bool = event['restricted']
- created_at: datetime = event['createdAt']
- updated_at: datetime = event['updatedAt']
- neg_risk: bool = event['negRisk'] # whether there are multiple markets whose No's resolve to a single other yes
- enable_neg_risk: bool = event['enableNegRisk']
- markets: list = event['markets'] # what markets are within the event
- tags: list = event['tags'] # what tags the event has
- series: list = event['series'] # NULLABLE what series the event belongs to
- series_slug: str = event['seriesSlug'] # NULLABLE slug of the series the event belongs to
- start_time: datetime = event['startTime'] # NULLABLE when the market "starts"
- category: str = event['category']
- subcategory: str = event['subcategory']
- comments_enabled:bool = event['commentsEnabled']


our data:
- last_update_time: datetime -> when we last updated this event in our database

In [37]:
from collections import defaultdict
key_counter = defaultdict(int)
for event in events:
    for key in event.keys():
        key_counter[key] += 1

In [75]:
# for event in events:
#     # if 'parentEventId' in event:
#     #     print(event['parentEventId'])
#     #     print(event)
#     #     break
#     # if 'finishedTimestamp' in event:
#         print("TICK", event.get('ticker'))
#         print("CD", event.get('creationDate'))
#         print("SD", event.get('startDate'))
#         print("ST", event.get('startTime'))
#         print("FIN", event.get('finishedTimestamp'))
#         print("ED", event.get('endDate'))
#         print("CT", event.get('closedTime'))
#         print("UA", event.get('updatedAt'))
#         print()
        

In [97]:
tag_key_count = defaultdict(int)
for event in events:
    if 'tags' in event:
        for tag in event['tags']:
            for key in tag.keys():
                tag_key_count[key] += 1

In [99]:
tag

{'id': '100199',
 'label': 'Senate',
 'slug': 'senate-elections',
 'forceShow': False,
 'updatedAt': '2024-11-04T21:50:59.684717Z'}

In [103]:
for event in events:
    if 'series' in event:
        print(len(event['series']))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [98]:
tag_key_count

defaultdict(int,
            {'id': 388565,
             'label': 388565,
             'slug': 388565,
             'forceShow': 338647,
             'updatedAt': 254276,
             'publishedAt': 164435,
             'createdAt': 381065,
             'updatedBy': 128721,
             'forceHide': 38041,
             'createdBy': 37249,
             'isCarousel': 106898})

In [38]:
key_counter

defaultdict(int,
            {'id': 72413,
             'ticker': 72411,
             'slug': 72413,
             'title': 72413,
             'description': 71731,
             'resolutionSource': 58172,
             'startDate': 72396,
             'creationDate': 72328,
             'endDate': 72273,
             'image': 72413,
             'icon': 72413,
             'active': 72413,
             'closed': 72413,
             'archived': 72405,
             'new': 72375,
             'featured': 72389,
             'restricted': 72405,
             'liquidity': 25281,
             'volume': 70325,
             'openInterest': 70790,
             'sortBy': 16714,
             'category': 2837,
             'published_at': 5089,
             'createdAt': 72413,
             'updatedAt': 72358,
             'competitive': 25471,
             'volume24hr': 5393,
             'volume1wk': 53010,
             'volume1mo': 53022,
             'volume1yr': 53022,
             'liquidityAm

In [65]:
mkey_counter = defaultdict(int)
i = 0
for event in events:
    # try:
    #     if event['cyom']:
    #         print(event)
    #         break
    # except Exception as e:
    #     # print(e)
    #     # print(event)
    #     # break
    #     pass
    if 'markets' not in event:
        continue
    
    for market in event['markets']:
        for key in market.keys():
            mkey_counter[key] += 1
        
        i += 1
            
    

In [66]:
i

164449

In [78]:
market['marketType']

'normal'

In [88]:
market['acceptingOrders']

False

In [92]:
for event in events:
    if 'markets' in event:
        for market in event['markets']:
            if 'gameId' in market:
                print(market['sportsMarketType'])
                print(market)
                break

totals
{'id': '528182', 'question': 'Over vs Under Line: 215.5', 'conditionId': '0xb0a5445d07cfd175fe0d840b3d0b59f832668a70a7773f654f2202265710255e', 'slug': 'nba-bos-mia-2025-03-14-total-215pt5', 'resolutionSource': 'https://www.nba.com/', 'endDate': '2025-03-21T23:00:00Z', 'liquidity': '0', 'startDate': '2025-03-14T18:14:51.025966Z', 'fee': '20000000000000000', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/super+cool+basketball+in+red+and+blue+wow.png', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/super+cool+basketball+in+red+and+blue+wow.png', 'description': 'In the upcoming NBA game, scheduled for March 14 at 7:00PM ET:\nIf the total score of the game is less than or equal to the line 215, the market will resolve to "Under".\nElse, the market will resolve to "Over".\nIf the game is canceled entirely, with no make-up game, this market will resolve 50-50.', 'outcomes': '["Over", "Under"]', 'outcomePrices': '["0", "1"]', 'active': True, 'closed': True

KeyError: 'sportsMarketType'

In [63]:
mkey_counter

defaultdict(int,
            {'id': 164449,
             'question': 164449,
             'conditionId': 164449,
             'slug': 164449,
             'resolutionSource': 129878,
             'endDate': 163713,
             'category': 4353,
             'liquidity': 58764,
             'startDate': 160702,
             'fee': 25816,
             'image': 164381,
             'icon': 164156,
             'description': 164449,
             'outcomes': 164449,
             'outcomePrices': 164319,
             'volume': 148641,
             'active': 164449,
             'marketType': 10072,
             'closed': 164449,
             'marketMakerAddress': 164449,
             'updatedBy': 5544,
             'createdAt': 164449,
             'updatedAt': 164429,
             'closedTime': 164066,
             'wideFormat': 25826,
             'new': 164418,
             'sentDiscord': 20516,
             'featured': 148810,
             'submitted_by': 130600,
             'twitterC

In [61]:
print(event.keys())

dict_keys(['id', 'ticker', 'slug', 'title', 'description', 'resolutionSource', 'startDate', 'creationDate', 'endDate', 'image', 'icon', 'active', 'closed', 'archived', 'new', 'featured', 'restricted', 'liquidity', 'volume', 'openInterest', 'sortBy', 'category', 'published_at', 'createdAt', 'updatedAt', 'competitive', 'volume24hr', 'volume1wk', 'volume1mo', 'volume1yr', 'liquidityAmm', 'liquidityClob', 'commentCount', 'tags', 'cyom', 'closedTime', 'showAllOutcomes', 'showMarketImages', 'enableNegRisk', 'negRiskAugmented', 'pendingDeployment', 'deploying'])


In [29]:

len(json.loads(event['markets'][0]['clobTokenIds']))

2

In [20]:
len(events)

4242

In [7]:
events[-1]

{'id': '82959',
 'ticker': 'btc-updown-15m-1763397000',
 'slug': 'btc-updown-15m-1763397000',
 'title': 'Bitcoin Up or Down - November 17, 11:30AM-11:45AM ET',
 'description': 'This market will resolve to "Up" if the Bitcoin price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the BTC/USD data stream available at https://data.chain.link/streams/btc-usd.\nPlease note that this market is about the price according to Chainlink data stream BTC/USD, not according to other sources or spot markets.',
 'resolutionSource': 'https://data.chain.link/streams/btc-usd',
 'startDate': '2025-11-16T16:39:35.450255Z',
 'creationDate': '2025-11-16T16:39:35.450252Z',
 'endDate': '2025-11-17T16:45:00Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png',
 'icon': 'https://polymarket-

### Data from Markets

In [4]:
events[-800]['markets'][0]

{'id': '685617',
 'question': "California Merced Bobcats vs. Saint Mary's Gaels",
 'conditionId': '0x4d99217999cd72a6e50566193b8f3690008d4386814257d67e206a2365506382',
 'slug': 'cbb-mrcd-stmry-2025-11-22',
 'resolutionSource': 'https://www.ncaa.com/',
 'endDate': '2025-11-23T03:30:00Z',
 'liquidity': '30.9997',
 'startDate': '2025-11-16T15:02:10.880392Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ncaa-c14995df96.png',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ncaa-c14995df96.png',
 'description': 'In the upcoming CBB game, scheduled for November 22 at 10:30 PM ET:\n\nIf the California Merced Bobcats win, the market will resolve to "California Merced Bobcats".\n\nIf the Saint Mary\'s Gaels win, the market will resolve to "Saint Mary\'s Gaels".\n\nIf the game is postponed, this market will remain open until the game has been completed.\n\nIf the game is canceled entirely, with no make-up game, this market will resolve 50-50.\n\nThe result will b

id
string
​
question
string | null
​
conditionId
string
​
slug
string | null
​
twitterCardImage
string | null
​
resolutionSource
string | null
​
endDate
string<date-time> | null
​
category
string | null
​
ammType
string | null
​
liquidity
string | null
​
sponsorName
string | null
​
sponsorImage
string | null
​
startDate
string<date-time> | null
​
xAxisValue
string | null
​
yAxisValue
string | null
​
denominationToken
string | null
​
fee
string | null
​
image
string | null
​
icon
string | null
​
lowerBound
string | null
​
upperBound
string | null
​
description
string | null
​
outcomes
string | null
​
outcomePrices
string | null
​
volume
string | null
​
active
boolean | null
​
marketType
string | null
​
formatType
string | null
​
lowerBoundDate
string | null
​
upperBoundDate
string | null
​
closed
boolean | null
​
marketMakerAddress
string
​
createdBy
integer | null
​
updatedBy
integer | null
​
createdAt
string<date-time> | null
​
updatedAt
string<date-time> | null
​
closedTime
string | null
​
wideFormat
boolean | null
​
new
boolean | null
​
mailchimpTag
string | null
​
featured
boolean | null
​
archived
boolean | null
​
resolvedBy
string | null
​
restricted
boolean | null
​
marketGroup
integer | null
​
groupItemTitle
string | null
​
groupItemThreshold
string | null
​
questionID
string | null
​
umaEndDate
string | null
​
enableOrderBook
boolean | null
​
orderPriceMinTickSize
number | null
​
orderMinSize
number | null
​
umaResolutionStatus
string | null
​
curationOrder
integer | null
​
volumeNum
number | null
​
liquidityNum
number | null
​
endDateIso
string | null
​
startDateIso
string | null
​
umaEndDateIso
string | null
​
hasReviewedDates
boolean | null
​
readyForCron
boolean | null
​
commentsEnabled
boolean | null
​
volume24hr
number | null
​
volume1wk
number | null
​
volume1mo
number | null
​
volume1yr
number | null
​
gameStartTime
string | null
​
secondsDelay
integer | null
​
clobTokenIds
string | null
​
disqusThread
string | null
​
shortOutcomes
string | null
​
teamAID
string | null
​
teamBID
string | null
​
umaBond
string | null
​
umaReward
string | null
​
fpmmLive
boolean | null
​
volume24hrAmm
number | null
​
volume1wkAmm
number | null
​
volume1moAmm
number | null
​
volume1yrAmm
number | null
​
volume24hrClob
number | null
​
volume1wkClob
number | null
​
volume1moClob
number | null
​
volume1yrClob
number | null
​
volumeAmm
number | null
​
volumeClob
number | null
​
liquidityAmm
number | null
​
liquidityClob
number | null
​
makerBaseFee
integer | null
​
takerBaseFee
integer | null
​
customLiveness
integer | null
​
acceptingOrders
boolean | null
​
notificationsEnabled
boolean | null
​
score
integer | null
​
imageOptimized
object
Show child attributes
​
iconOptimized
object
Show child attributes
​
events
object[]
Show child attributes
​
categories
object[]
Show child attributes
​
tags
object[]
Show child attributes
​
creator
string | null
​
ready
boolean | null
​
funded
boolean | null
​
pastSlugs
string | null
​
readyTimestamp
string<date-time> | null
​
fundedTimestamp
string<date-time> | null
​
acceptingOrdersTimestamp
string<date-time> | null
​
competitive
number | null
​
rewardsMinSize
number | null
​
rewardsMaxSpread
number | null
​
spread
number | null
​
automaticallyResolved
boolean | null
​
oneDayPriceChange
number | null
​
oneHourPriceChange
number | null
​
oneWeekPriceChange
number | null
​
oneMonthPriceChange
number | null
​
oneYearPriceChange
number | null
​
lastTradePrice
number | null
​
bestBid
number | null
​
bestAsk
number | null
​
automaticallyActive
boolean | null
​
clearBookOnStart
boolean | null
​
chartColor
string | null
​
seriesColor
string | null
​
showGmpSeries
boolean | null
​
showGmpOutcome
boolean | null
​
manualActivation
boolean | null
​
negRiskOther
boolean | null
​
gameId
string | null
​
groupItemRange
string | null
​
sportsMarketType
string | null
​
line
number | null
​
umaResolutionStatuses
string | null
​
pendingDeployment
boolean | null
​
deploying
boolean | null
​
deployingTimestamp
string<date-time> | null
​
scheduledDeploymentTimestamp
string<date-time> | null
​
rfqEnabled
boolean | null
​
eventStartTime
string<date-time> | null

In [19]:
events[-800]['markets']

[{'id': '687727',
  'question': 'Ethereum Up or Down - November 18, 2:00PM-2:15PM ET',
  'conditionId': '0xbd9d820b6403c42bba61e53f18435918b5e4f0e34f0314c7ae38600c64c50fa8',
  'slug': 'eth-updown-15m-1763492400',
  'resolutionSource': 'https://data.chain.link/streams/eth-usd',
  'endDate': '2025-11-18T19:15:00Z',
  'liquidity': '83029.526',
  'startDate': '2025-11-17T19:02:53.310584Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg',
  'description': 'This market will resolve to "Up" if the Ethereum price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the ETH/USD data stream available at https://data.chain.link/streams/eth-usd.\nPlease note that this market is about the price a

- id: str = market['id']
- question: str = market['question']
- slug: str = market['slug']
- title: str = market['title']
- description: str = market['description']
- resolution_source: str = market['resolutionSource']